In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.concat(map(pd.read_csv, 
                   ['walmart.csv', 
                    'target.csv',
                    'amazon.csv',
                    'costco.csv',
                    'kroger.csv']),ignore_index = True)

df.loc[:,'tokens_final'] = \
df.loc[:,'tokens_final'].apply(lambda x: literal_eval(x))

def replace_in_list(lis, old, new):
    for i in range(len(lis)):
        if lis[i] == old:
            lis[i] = new
    return(lis)

df['tokens_final'] = \
df['tokens_final'].map(lambda x: replace_in_list(x,"employee", "associate"))

df['tokens_final'] = \
df['tokens_final'].map(lambda x: replace_in_list(x,"guest", "customer"))

df['tokens_final'] = \
df['tokens_final'].map(lambda x: replace_in_list(x,"consumer", "customer"))

remove_words =['performance','officer','value','award','amount',
               'cash','chairman','vice','option','president',
               'base','inc.','grant','voting','election','unit',
               'audit','benefit','date','service','management',
               'include','number','name','person','proposal',
               'section','report','cost','receive','pension',
               'rate','interest','fuel','rate','serve','sale','pension']

df["tokens_final"] = \
    df["tokens_final"].map(lambda x: \
    [word for word in x if word.lower() not in remove_words])

In [3]:
df

,year,company,tokens_final
0,2010,Walmart,"[street, website, www.walmartstores.com, notic..."
1,2010,Walmart,"[wal-mart, store, street, website, www.walmart..."
2,2010,Walmart,"[nonqualified, compensation, potential, paymen..."
3,2010,Walmart,"[compensation, store, compensation, amend, jan..."
4,2010,Walmart,"[nominee, statement, company, ratification, ap..."
...,...,...,...
3710,2019,Kroger,"[company, termination, cause, treatment, provi..."
3711,2019,Kroger,"[purpose, approve, company, provide, provision..."
3712,2019,Kroger,"[security, restriction, represent, book, accou..."
3713,2019,Kroger,"[case, jurisdiction, approve, discretion, prov..."


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [5]:
#ran once to write files

# import os
# os.mkdir("temp_text")

# companies = df["company"].drop_duplicates().values

# for company in companies:
#     os.mkdir("temp_text\\" + company)
    
# for index, row in df.iterrows():
#     company = row["company"]
#     year = row["year"]
#     text = ' '.join(row["tokens_final"])
    
#     with open(f"temp_text\\{company}\\{year}_{company}_{index}.txt", 'w', encoding='utf8') as f:
#         f.write(text)

In [6]:
import os, gensim

def iter_documents(top_directory):
    """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
    for root, dirs, files in os.walk(top_directory):
        for file in filter(lambda file: file.endswith('.txt'), files):
            document = open(os.path.join(root, file), encoding='utf8').read() # read the entire document, as one big string
            yield gensim.utils.tokenize(document, lower=True) # or whatever tokenization suits you

class MyCorpus(object):
    def __init__(self, top_dir):
        self.top_dir = top_dir
        self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
        self.dictionary.filter_extremes(no_below=1, keep_n=30000) # check API docs for pruning params

    def __iter__(self):
        for tokens in iter_documents(self.top_dir):
            yield self.dictionary.doc2bow(tokens)

In [7]:
walmart_corpus = MyCorpus('temp_text/walmart')
amazon_corpus  = MyCorpus('temp_text/amazon')
costco_corpus  = MyCorpus('temp_text/costco')
target_corpus  = MyCorpus('temp_text/target')
kroger_corpus  = MyCorpus('temp_text/kroger')

In [8]:
from gensim.models import LdaModel
ntopic = 10

lda_walmart = LdaModel(walmart_corpus, num_topics=ntopic, id2word=walmart_corpus.dictionary)
lda_amazon  = LdaModel(amazon_corpus , num_topics=ntopic, id2word=amazon_corpus.dictionary)
lda_costco  = LdaModel(costco_corpus , num_topics=ntopic, id2word=costco_corpus.dictionary)
lda_target  = LdaModel(target_corpus , num_topics=ntopic, id2word=target_corpus.dictionary)
lda_kroger  = LdaModel(kroger_corpus , num_topics=ntopic, id2word=kroger_corpus.dictionary)

In [9]:
names = [("Walmart", walmart_corpus, lda_walmart), 
         ("Amazon", amazon_corpus, lda_amazon),
         ("Costco", costco_corpus, lda_costco), 
         ("Target", target_corpus, lda_target), 
         ("Kroger", kroger_corpus, lda_kroger)]

In [10]:
import itertools
names_prod = list(itertools.product(names, names))

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [12]:
from collections import OrderedDict

def average_topic(flat_list):

    d = OrderedDict()
    for prob, topic in flat_list:
        d.setdefault(topic, []).append(prob)

    d = [(sum(v) / len(v), k) for k, v in d.items()]
    
    return(d)

In [16]:
for i in range(ntopic):
    pp.pprint(lda_walmart.show_topic(topicid=i))
    print()

[   ('member', 0.020815318),
    ('experience', 0.015948776),
    ('cngc', 0.014865066),
    ('business', 0.011187044),
    ('incentive', 0.010436272),
    ('corporation', 0.009863281),
    ('review', 0.009304732),
    ('governance', 0.009019247),
    ('group', 0.008369606),
    ('rule', 0.008257974)]

[   ('column', 0.017674008),
    ('equity', 0.0146302385),
    ('incentive', 0.010269762),
    ('payment', 0.009645595),
    ('information', 0.00938826),
    ('table', 0.008672465),
    ('review', 0.0083433),
    ('douglas', 0.008292766),
    ('rule', 0.00827823),
    ('market', 0.007920543)]

[   ('transaction', 0.016856004),
    ('member', 0.014321849),
    ('review', 0.012210391),
    ('policy', 0.011638354),
    ('goal', 0.011564127),
    ('incentive', 0.01017085),
    ('cngc', 0.009424425),
    ('accountant', 0.009326033),
    ('recipient', 0.009256202),
    ('target', 0.008195261)]

[   ('transaction', 0.014363549),
    ('business', 0.0126234),
    ('material', 0.01241821),
    ('a

In [27]:
names_prod[5]

(('Amazon',
  <gensim.models.ldamodel.LdaModel at 0x175fb21e390>),
 ('Walmart',
  <gensim.models.ldamodel.LdaModel at 0x175faec3f28>))

In [22]:
import operator

for index, pair in enumerate(names_prod):
    model_txt  = pair[0][0]
    corpus_txt = pair[1][0]
    
    model  = pair[0][2]
    corpus = pair[1][1]
    
    print(f"{model_txt} model applied to {corpus_txt} documents")
    tag = [model.get_document_topics(item) for item in corpus]
    tag = [tup[::-1] for tup in flatten(tag)]
    
    topic_avg = average_topic(tag)
    topic_avg = sorted(topic_avg, key = lambda x: x[1])
    
    pp.pprint(topic_avg)
    print()

Walmart model applied to Walmart documents
[   (0.5749485726530353, 0),
    (0.659026976513886, 1),
    (0.4697784031475229, 2),
    (0.4411118731981045, 3),
    (0.4724532369684601, 4),
    (0.4844073406833259, 5),
    (0.36388357516636355, 6),
    (0.4208052073600156, 7),
    (0.5742469740051969, 8),
    (0.5099018941637113, 9)]

Walmart model applied to Amazon documents
[   (0.4584708307267499, 0),
    (0.1604150372640744, 1),
    (0.1915824999711637, 2),
    (0.21484494816292735, 3),
    (0.13429291695881057, 4),
    (0.15634363830562623, 5),
    (0.0771658681333065, 6),
    (0.06407589491988931, 7),
    (0.27291325400903743, 8),
    (0.2841162194811444, 9)]

Walmart model applied to Costco documents
[   (0.46902829638984, 0),
    (0.19138230993239969, 1),
    (0.15899273125926056, 2),
    (0.1698810890163869, 3),
    (0.18826918323834738, 4),
    (0.20826677218105438, 5),
    (0.05865619461983442, 6),
    (0.0804566657791535, 7),
    (0.23010121727068172, 8),
    (0.30179525658030

IndexError: index 2317 is out of bounds for axis 1 with size 2317